In [1]:
import numpy as np
import itertools
def logical_error_rate(p_phy, d):
    if d%2 == 1:
        return 0.1 * (p_phy / 0.01)**(0.5 * d + 0.5)
    else:
        return 0.05 * (d*d)/(d-1)/(d-1) * (p_phy / 0.01)**(0.5 * d)
    
def logical_error_retangle(dz, dx, p_phy):
    logical_error_z = 0.5 * (dx/dz) * logical_error_rate(p_phy, dz)
    logical_error_x = 0.5 * (dz/dx) * logical_error_rate(p_phy, dx)
    return logical_error_z, logical_error_x

def distillation_code_distance(eps, p_phy):
    for d in range(3,50,2):
        if 6.5 * d * logical_error_rate(p_phy, d) < eps:
            return d

# def logical_error_rate(code_distance, physical_error_rate=0.001):
#     return 0.1*(physical_error_rate/0.01)**((code_distance+1)/2)

# def code_distance(logical_error_rate, physical_error_rate=0.001):
#     exponent = np.log(logical_error_rate*10)/np.log(physical_error_rate/0.01)
#     return max(int(np.ceil(exponent))*2-1, 1)

def search_t_protocol(target_error_rate, p_phy):
    t_error_range = []
    initial_t_error_range = []
    if p_phy == 0.001:
        eps = [0.0004, 8e-05, 7e-06, 5e-06, 3e-06, 2e-07, 3e-08, 1e-08, 6e-09, 2e-09, 1e-09, 
             3.8850000000000016e-10, 3.304716553287983e-10, 1.2955e-12, 1.2178655692729767e-12, 4.405000000000001e-14,
             3.563400713436386e-14, 4.815000000000002e-15, 3.9081789802289295e-15, 1.2225000000000002e-15, 1.1252984389348027e-15, 
             1.3475000000000002e-16, 5.0750000000000023e-17, 4.03781224489796e-17]
        volume = [1000, 4000, 5000, 6000, 8000, 20000, 30000, 40000, 50000, 60000, 100000, 
             2199074, 2523632, 4685822, 5216768, 5827026, 6438000, 7131094, 7827712, 8609258, 9397136, 10272750, 10432750, 11317504]
    elif p_phy == 0.0005:
        eps=[1e-5, 8e-7,3e-07, 2e-07, 1e-08, 1e-09, 4e-11, 3.7260546875000005e-12, 3.2233414127423824e-12, 1.186650390625e-12, 
             1.1583162379535146e-12,2.9983642578125002e-15, 2.91156640625e-15, 3.871166992187501e-17, 3.3973240312071337e-17, 
             7.554645996093752e-20,6.170946930280958e-20]
        volume=[2000, 3500, 4000, 4500, 8000.0, 10000.0, 20000.0, 1621006, 1888000, 2185074, 2509632, 3688250.0, 4144784.0, 
                4645822.0, 5176768.0,7051094.0, 7747712.0]
    
    for i, t_error in enumerate(eps): 
        s = 8 * t_error ** 2
        if s < 1.5 * target_error_rate:
            t_error_range.append([eps[i],volume[i]])
        if len(t_error_range) >= 6:
            break
    for i, t_error in enumerate(eps): 
        s = t_error
        if s < 1.5 * target_error_rate:
            initial_t_error_range.append([eps[i],volume[i]])
        if len(initial_t_error_range) >= 6:
            break
    return t_error_range, initial_t_error_range #length = 6


def MEK(d, t_error, rl_1_error, ml_error, t_volume, rl_1_volume, ml_volume, p_phy): # Campbell-1603.04230
    # ml_error 输入态的error
    # rl_1_error R l-1 门的error（包含修正）
    pass_rate = 1 - 8 * t_error - 2 * ml_error - 0.5 * rl_1_error
    pass_rate += 96 * logical_error_rate(p_phy, d)
    
    error_rate = 8 * t_error ** 2 + ml_error ** 2 + rl_1_error / 4
    error_rate += 0.5 * logical_error_rate(p_phy, d)
    
    
    depth = 20 * d
    footprint = 2 * 16 * d ** 2
    
    volume = (footprint * depth + 8 * t_volume + 2 * ml_volume + rl_1_volume) / pass_rate / 2

    return error_rate, volume

def mek_level_one(d, t_error, initial_t_error, t_volume,initial_t_volume, ll, p_phy): # d, t_error, ll>=4
    ml_error = 7 / 15 * p_phy
    ml_volume = 0
    rl_1_error = initial_t_error
    rl_1_volume = initial_t_volume
    error_rates = [initial_t_error]
    volumes = [initial_t_volume]
    for l in range(4, ll+1):
        error_rate, volume = MEK(d, t_error, rl_1_error, ml_error, t_volume, rl_1_volume, ml_volume, p_phy)
        error_rates.append(error_rate)
        volumes.append(volume)
        rl_1_error = rl_1_error * 0.5 + error_rate
        rl_1_volume = rl_1_volume * 0.5 + volume
    return error_rates, volumes

def mek_level_two(d1,d2, t_error1, initial_t_error1, t_volume1,initial_t_volume1,t_error2, initial_t_error2, 
                  t_volume2, initial_t_volume2, ll, p_phy): 
    error_rates_one, volumes_one = mek_level_one(d1, t_error1, initial_t_error1, initial_t_volume1, t_volume1, 25, p_phy)#限定最大L=25
    error_rates = [initial_t_error2]
    volumes = [initial_t_volume2]
    rl_1_error = initial_t_error2
    rl_1_volume = initial_t_volume2
    for l in range(4, ll+1):
        ml_error = error_rates_one[l-3]
        ml_volume = volumes_one[l-3]
        error_rate, volume = MEK(d2, t_error2, rl_1_error, ml_error, t_volume2, rl_1_volume, ml_volume, p_phy)
        error_rates.append(error_rate)
        volumes.append(volume)
        rl_1_error = rl_1_error * 0.5 + error_rate
        rl_1_volume = rl_1_volume * 0.5 + volume
    return error_rates, volumes

def mek_level_three(d1,d2,d3, t_error1, initial_t_error1,t_volume1,initial_t_volume1,t_error2, initial_t_error2, 
                    t_volume2, initial_t_volume2,  t_error3, initial_t_error3, t_volume3, initial_t_volume3, ll, p_phy): 
    error_rates_two, volumes_two = mek_level_two(d1,d2, t_error1, initial_t_error1, initial_t_volume1, t_volume1,t_error2,
                                                 initial_t_error2, initial_t_volume2, t_volume2, 25, p_phy)#限定最大L=25
    error_rates = [initial_t_error3]
    volumes = [initial_t_volume3]
    rl_1_error = initial_t_error3
    rl_1_volume = initial_t_volume3
    for l in range(4, ll+1):
        ml_error = error_rates_two[l-3]
        ml_volume = volumes_two[l-3]
        error_rate, volume = MEK(d2, t_error2, rl_1_error, ml_error, t_volume2, rl_1_volume, ml_volume, p_phy)
        error_rates.append(error_rate)
        volumes.append(volume)
        rl_1_error = rl_1_error * 0.5 + error_rate
        rl_1_volume = rl_1_volume * 0.5 + volume
    return error_rates, volumes

def search_mek_level_two(ll, target_error_rate, p_phy):#搜索t_error，d用梯度下降
    parameters = ["fail",float("inf")] 
    t_error_range2, initial_t_error_range2 = search_t_protocol(target_error_rate, p_phy)
    t_error_range1, initial_t_error_range1 = search_t_protocol(target_error_rate**0.5, p_phy)
    for t_error1, t_volume1 in t_error_range1:
        for initial_t_error1, initial_t_volume1 in initial_t_error_range1:
            for t_error2, t_volume2 in t_error_range2:
                for initial_t_error2, initial_t_volume2 in initial_t_error_range2:
                    d_target = [41,41] #d1,d2
                    d_step = [1,1] #搜索步长
                    for i in range(len(d_target)):
                        d_range0 = [41,41]
                        for j in range(d_target[i]//d_step[i]):
                            d1,d2= d_range0
                            error_rates, volumes = mek_level_two(d1,d2, t_error1, initial_t_error1, t_volume1,initial_t_volume1,
                                                                 t_error2, initial_t_error2, t_volume2, initial_t_volume2, ll, p_phy)
                            if error_rates[-1] < target_error_rate:
                                d_target[i] = d_range0[i]
                                d_range0[i] -= d_step[i]
                            else:
                                break
                    d1,d2 = d_target
                    d_set = list(itertools.product([d1,d1+2],[d2,d2+2]))
                    for dd1, dd2 in d_set:
                        error_rates, volumes = mek_level_two(dd1,dd2, t_error1, initial_t_error1, t_volume1,initial_t_volume1,
                                                         t_error2, initial_t_error2, t_volume2, initial_t_volume2, ll, p_phy)
                        if error_rates[-1] < target_error_rate and volumes[-1] < parameters[-1]:
                                parameters = [dd1,dd2, t_error1, initial_t_error1, t_volume1,initial_t_volume1,
                                              t_error2, initial_t_error2, t_volume2, initial_t_volume2, error_rates[-1], volumes[-1]]
    return parameters
    
    
    
def search_mek_level_three(ll, target_error_rate, p_phy):#搜索t_error，d用梯度下降
    parameters = ["fail",float("inf")] 
    t_error_range3, initial_t_error_range3 = search_t_protocol(target_error_rate, p_phy)
    t_error_range2, initial_t_error_range2 = search_t_protocol(target_error_rate**0.5, p_phy)
    t_error_range1, initial_t_error_range1 = search_t_protocol((target_error_rate**0.5)**0.5, p_phy)
    for t_error1, t_volume1 in t_error_range1:
        for initial_t_error1, initial_t_volume1 in initial_t_error_range1:
            for t_error2, t_volume2 in t_error_range2:
                for initial_t_error2, initial_t_volume2 in initial_t_error_range2:
                    for t_error3, t_volume3 in t_error_range3:
                        for initial_t_error3, initial_t_volume3 in initial_t_error_range3:
                            d_target = [41,41,41] #d1,d2
                            d_step = [1,1,1] #搜索步长
                            for i in range(len(d_target)):
                                d_range0 = [41,41,41]
                                for j in range(d_target[i]//d_step[i]):
                                    d1,d2,d3= d_range0
                                    error_rates, volumes = mek_level_three(d1,d2,d3, t_error1, initial_t_error1,t_volume1,initial_t_volume1,
                                                                           t_error2, initial_t_error2, t_volume2, initial_t_volume2,  t_error3, 
                                                                           initial_t_error3, t_volume3, initial_t_volume3, ll, p_phy)
                                    if error_rates[-1] < target_error_rate:
                                        d_target[i] = d_range0[i]
                                        d_range0[i] -= d_step[i]
                                    else:
                                        break
                            d1,d2,d3 = d_target
                            d_set = list(itertools.product([d1,d1+2],[d2,d2+2],[d3,d3+2]))
                            for dd1, dd2, dd3 in d_set:
                                error_rates, volumes = mek_level_three(dd1,dd2,dd3, t_error1, initial_t_error1,t_volume1,initial_t_volume1,
                                                                           t_error2, initial_t_error2, t_volume2, initial_t_volume2,  t_error3, 
                                                                           initial_t_error3, t_volume3, initial_t_volume3, ll, p_phy)
                                if error_rates[-1] < target_error_rate and volumes[-1] < parameters[-1]:
                                        parameters = [dd1,dd2,dd3, t_error1, initial_t_error1,t_volume1,initial_t_volume1,
                                                                           t_error2, initial_t_error2, t_volume2, initial_t_volume2,  t_error3, 
                                                                           initial_t_error3, t_volume3, initial_t_volume3, error_rates[-1], volumes[-1]]
    return parameters

In [25]:
np.pi/2**12,np.pi/2**7, np.pi/2**9, np.pi/2**25

(0.0007669903939428206,
 0.02454369260617026,
 0.006135923151542565,
 9.362675707309822e-08)

In [2]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-12, p_phy = 1e-3)[-2:]
    print(l,error_rates, volumes)

4 9.323070074911609e-13 5935354.856282866
5 9.92773809701025e-13 8319532.407568745
6 8.968789864090455e-13 11049669.005112411
7 8.61451692858505e-13 13574631.156403974
8 9.065540551914748e-13 16226986.134095533
9 9.463197979874926e-13 19006383.228704613
10 9.806417659969662e-13 21912884.844523806
11 9.978352154165215e-13 25126542.21948627
12 9.90716428482739e-13 28878042.641512465
13 8.218726150788152e-13 34081834.639264464
14 8.420814884802837e-13 37743974.27925465
15 8.588375182183858e-13 41534091.98287816
16 8.726061627339391e-13 45452288.09274603
17 8.838350390777448e-13 49498663.03155925
18 8.929341527211688e-13 53673317.30211494
19 9.002669524310033e-13 57976351.48725948
20 9.061481099271291e-13 62407866.24981598
21 9.108452640971737e-13 66967962.332503565
22 9.145829105021532e-13 71656740.55785938
23 9.17547268149807e-13 76474301.82816969
24 9.198913994777064e-13 81420747.12541339
25 9.217401570823107e-13 86496177.51121926


In [2]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-12, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)


4 9.103414907426698e-13 2809525.7219301634
5 7.978610791141012e-13 3747955.3615733115
6 8.72819805072941e-13 4705375.713829362
7 9.401723887777299e-13 5693789.864059412
8 9.991181290473867e-13 6713205.392127753
9 9.504721220991922e-13 7780644.409942243
10 9.911341894213873e-13 8888114.277798833
11 9.99476133560076e-13 10765267.065367242
12 8.769753587928336e-13 13010426.934259374
13 8.775312200478859e-13 14549345.487878555
14 7.161694015457578e-13 14686363.833560463
15 7.199895301729789e-13 16112718.758389398
16 7.230709144743589e-13 17583715.776902262
17 7.255442257973096e-13 19099366.257820893
18 7.275209463005363e-13 20659681.572888587
19 7.290948075421507e-13 22264673.09685872
20 7.303437043060949e-13 23914352.20748187
21 7.313317582324339e-13 25608730.28549269
22 7.321113379146911e-13 27347818.71459747
23 7.327249289171489e-13 29131628.88146278
24 7.332068028691749e-13 30960172.175705366
25 7.335844694318611e-13 32833459.989883486


In [4]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-9, p_phy = 1e-3)[-2:]
    print(l,error_rates, volumes)

4 9.992660490449385e-10 1285910.6893036363
5 9.772063250050622e-10 1980826.2543885268
6 9.579182826522613e-10 2796185.935368079
7 9.982764333535528e-10 3629765.877249147
8 8.811955312564242e-10 4928564.803230946
9 8.401265240781025e-10 5861464.967493751
10 8.594849879258826e-10 6826484.612840274
11 8.718356109954797e-10 7823910.669527167
12 8.841551925263459e-10 8853626.285251355
13 8.939929495374237e-10 9915815.223166836
14 9.018209192406063e-10 11010478.80332807
15 9.080297535503431e-10 12137618.349662669
16 9.129401197233607e-10 13297235.191102693
17 9.168133952460399e-10 14489330.661977015
18 9.198613375615751e-10 15713906.101947028
19 9.222545954305298e-10 16970962.855677065
20 9.241300494873755e-10 18260502.272366956
21 9.255970425996769e-10 19582525.70522925
22 9.267426003739583e-10 20937034.51096277
23 9.276357596044135e-10 22324030.049253076
24 9.283311256865835e-10 23743513.682316355
25 9.2887177470275e-10 25195486.77449408


In [5]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-9, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)

4 8.739023297264202e-10 498900.15626140457
5 9.9796922284615e-10 907058.9396785971
6 9.642009177563714e-10 1296825.738581559
7 9.232012047711445e-10 1657737.6658366732
8 9.969501590429027e-10 2025624.7135735664
9 8.920775142515376e-10 2994884.6052382933
10 9.045287026260405e-10 3544217.3316796795
11 9.138135739394673e-10 4123071.473634731
12 9.207384001036253e-10 4731454.351166841
13 9.259036376043151e-10 5369373.288944169
14 9.297566944701545e-10 6036835.614606195
15 9.326310672991063e-10 6733848.657705364
16 9.347754134691851e-10 7460419.7490547495
17 9.36375171928217e-10 8216556.2203572085
18 3.653706139927807e-10 8958674.336425738
19 3.6699877046649127e-10 9620655.652996035
20 3.683266386114263e-10 10294571.115581932
21 3.6940262638129546e-10 10980417.081850285
22 3.7026969546209555e-10 11678189.913070444
23 3.7096506342010075e-10 12387885.973729543
24 3.715203931939213e-10 13109501.631191164
25 3.719622457378285e-10 13843033.25539881


In [6]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-15, p_phy = 1e-3)[-2:]
    print(l,error_rates, volumes)

4 fail inf
5 fail inf
6 fail inf
7 fail inf
8 fail inf
9 fail inf
10 fail inf
11 fail inf
12 fail inf
13 fail inf
14 fail inf
15 fail inf
16 fail inf
17 fail inf
18 fail inf
19 fail inf
20 fail inf
21 fail inf
22 fail inf
23 fail inf
24 fail inf
25 fail inf


In [7]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-15, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)

4 fail inf
5 fail inf
6 fail inf
7 fail inf
8 fail inf
9 fail inf
10 fail inf
11 fail inf
12 fail inf
13 fail inf
14 fail inf
15 fail inf
16 fail inf
17 fail inf
18 fail inf
19 fail inf
20 fail inf
21 fail inf
22 fail inf
23 fail inf
24 fail inf
25 fail inf


In [8]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_three(ll = l, target_error_rate = 1e-15, p_phy = 1e-3)[-2:]
    print(l,error_rates, volumes)

4 8.439846065895109e-16 56079250.889510356
5 9.167009472781182e-16 104653646.21945538
6 9.8755574384119e-16 164613875.28093535
7 9.44991577426079e-16 236812709.33878466
8 9.99802687171487e-16 320669969.7709414
9 8.011490703740045e-16 421734168.0261643
10 8.615175472054175e-16 529444524.4642771
11 9.228234935250678e-16 648826569.9038798
12 9.827444497588717e-16 779900858.1630558
13 9.933617316428546e-16 923677867.2073131
14 8.067843604211631e-16 1101494111.773857
15 8.201500514311596e-16 1271433856.3818
16 8.319961451104251e-16 1453420056.9137862
17 8.42358804410346e-16 1647477761.8544402
18 8.513215095297482e-16 1853631946.1811016
19 8.589970720935495e-16 2071907512.3853543
20 8.655135392645863e-16 2302329291.360427
21 8.710037448768889e-16 2544922043.1701756
22 8.755979921374187e-16 2799710457.7158346
23 8.794192672199023e-16 3066719155.3159585
24 8.825804021995515e-16 3345972687.213625
25 8.851826758055333e-16 3637495536.023291


In [9]:
for l in range(4, 26):
    error_rates, volumes = search_mek_level_three(ll = l, target_error_rate = 1e-15, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)

4 5.06384800872203e-16 29193417.509963434
5 fail inf
6 3.679086187698823e-16 86251953.01811187
7 4.0201495404321215e-16 123781628.92048627
8 4.275996442214797e-16 167349470.05521846
9 4.467929705156482e-16 216964320.9774774
10 4.611924207519985e-16 272635021.9538027
11 4.719959902410594e-16 334370409.02519965
12 4.801021304790416e-16 402179314.11684453
13 4.861846853372178e-16 476070565.16729146
14 4.907490729806803e-16 556052986.2606946
15 4.941744076792488e-16 642135397.7525706
16 4.967450813632003e-16 734326616.3841646
17 4.986744436938205e-16 832635455.3833469
18 5.0012255866067e-16 937070724.5517141
19 5.012095203470718e-16 1047641230.3385625
20 5.02025439197367e-16 1164355775.9029362
21 5.026379318303348e-16 1287223161.165158
22 5.030977388760879e-16 1416252182.8493018
23 5.034429389978708e-16 1551451634.5179853
24 5.037021100995157e-16 1692830306.6007373
25 5.03896700899936e-16 1840396986.41705


In [10]:
for l in range(4, 25):
    error_rates, volumes = search_mek_level_two(ll = l, target_error_rate = 1e-13, p_phy = 5e-4)[-2:]
    print(l,error_rates, volumes)

4 7.756905163092865e-14 3992782.6647231868
5 9.62863244274412e-14 5191887.876813458
6 9.398478767020683e-14 6505455.3937657755
7 4.198536480956279e-14 8598982.338957548
8 4.449907149867788e-14 10149725.215688327
9 4.6353394598165306e-14 11762941.272689719
10 4.772093480316494e-14 13438647.517332021
11 4.872919712442532e-14 15176860.961630471
12 4.947235416694131e-14 16977598.62223125
13 5.00199449458463e-14 18840877.520403508
14 5.042330683638601e-14 20766714.682035487
15 5.0720330735772606e-14 22755127.1376333
16 5.09389760145739e-14 24806131.922321532
17 5.1099868266767745e-14 26919746.075844843
18 5.121821883176852e-14 29095986.642570242
19 5.130524287518075e-14 31334870.67148964
20 5.136920679685684e-14 33636415.21622254
21 5.141620168272261e-14 36000637.33501871
22 5.145071432119965e-14 38427554.0907608
23 5.147604866440263e-14 40917182.55096679
24 5.149463682500642e-14 43469539.78779233


In [2]:
#T decomposition
def T_decomposition(ll, p_phy, infidelity):
    if p_phy == 0.001:
        eps = [0.0004, 8e-05, 7e-06, 5e-06, 3e-06, 2e-07, 3e-08, 1e-08, 6e-09, 2e-09, 1e-09, 
                 3.8850000000000016e-10, 3.304716553287983e-10, 1.2955e-12, 1.2178655692729767e-12, 4.405000000000001e-14,
                 3.563400713436386e-14, 4.815000000000002e-15, 3.9081789802289295e-15, 1.2225000000000002e-15, 1.1252984389348027e-15, 
                 1.3475000000000002e-16, 5.0750000000000023e-17, 4.03781224489796e-17]
        volume = [1000, 4000, 5000, 6000, 8000, 20000, 30000, 40000, 50000, 60000, 100000, 
                 2199074, 2523632, 4685822, 5216768, 5827026, 6438000, 7131094, 7827712, 8609258, 9397136, 10272750, 10432750, 11317504]
    elif p_phy == 0.0005:
        eps=[1e-5, 8e-7,3e-07, 2e-07, 1e-08, 1e-09, 4e-11, 3.7260546875000005e-12, 3.2233414127423824e-12, 1.186650390625e-12, 
                 1.1583162379535146e-12,2.9983642578125002e-15, 2.91156640625e-15, 3.871166992187501e-17, 3.3973240312071337e-17, 
                 7.554645996093752e-20,6.170946930280958e-20]
        volume=[2000, 3500, 4000, 4500, 8000.0, 10000.0, 20000.0, 1621006, 1888000, 2185074, 2509632, 3688250.0, 4144784.0, 
                    4645822.0, 5176768.0,7051094.0, 7747712.0]

    #infidelity = 1e-13
    output1, output2 = 0, float("inf")
    for l in range(3, 30):
        t_count = int (3 * (l - 1 - np.log(np.pi)/np.log(2)))
        for i,e in enumerate(eps):
            if e * t_count +  (np.pi/2**l)**2 <= infidelity * 2: ### multiple 2 
                if t_count * volume[i] < output2:
                    output1, output2 = e * t_count, t_count * volume[i]
    return output1, output2
        
def T_decomposition_added(ll, p_phy, infidelity):
    if p_phy == 0.001:
        eps = [0.0004, 8e-05, 7e-06, 5e-06, 3e-06, 2e-07, 3e-08, 1e-08, 6e-09, 2e-09, 1e-09, 
                 3.8850000000000016e-10, 3.304716553287983e-10, 1.2955e-12, 1.2178655692729767e-12, 4.405000000000001e-14,
                 3.563400713436386e-14, 4.815000000000002e-15, 3.9081789802289295e-15, 1.2225000000000002e-15, 1.1252984389348027e-15, 
                 1.3475000000000002e-16, 5.0750000000000023e-17, 4.03781224489796e-17]
        volume = [1000, 4000, 5000, 6000, 8000, 20000, 30000, 40000, 50000, 60000, 100000, 
                 2199074, 2523632, 4685822, 5216768, 5827026, 6438000, 7131094, 7827712, 8609258, 9397136, 10272750, 10432750, 11317504]
    elif p_phy == 0.0005:
        eps=[1e-5, 8e-7,3e-07, 2e-07, 1e-08, 1e-09, 4e-11, 3.7260546875000005e-12, 3.2233414127423824e-12, 1.186650390625e-12, 
                 1.1583162379535146e-12,2.9983642578125002e-15, 2.91156640625e-15, 3.871166992187501e-17, 3.3973240312071337e-17, 
                 7.554645996093752e-20,6.170946930280958e-20]
        volume=[2000, 3500, 4000, 4500, 8000.0, 10000.0, 20000.0, 1621006, 1888000, 2185074, 2509632, 3688250.0, 4144784.0, 
                    4645822.0, 5176768.0,7051094.0, 7747712.0]

    #infidelity = 1e-13
    output1, output2 = 0, float("inf")
    for l in range(3, 30):
        t_count = int (3 * (l - 1 - np.log(np.pi)/np.log(2)))
        for i,e in enumerate(eps):
            if e * t_count +  (np.pi/2**l)**2 <= infidelity * (l-2): ### multiple l-2 
                if t_count * volume[i] < output2:
                    output1, output2 = e * t_count, t_count * volume[i]
    return output1, output2

In [24]:
np.log(np.pi*(1e12)**0.5)/np.log(2)

np.float64(21.583064698796495)

In [3]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(ll=l, p_phy=0.0005, infidelity=1e-12)
    print(l,"("+str(volumes)+")")

4 (213918500.0)
5 (213918500.0)
6 (213918500.0)
7 (213918500.0)
8 (213918500.0)
9 (213918500.0)
10 (213918500.0)
11 (213918500.0)
12 (213918500.0)
13 (213918500.0)
14 (213918500.0)
15 (213918500.0)
16 (213918500.0)
17 (213918500.0)
18 (213918500.0)
19 (213918500.0)
20 (213918500.0)
21 (213918500.0)
22 (213918500.0)
23 (213918500.0)
24 (213918500.0)
25 (213918500.0)


In [4]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(ll=l, p_phy=0.001, infidelity=1e-12)
    print(l,"("+str(volumes)+")")

4 (413603452)
5 (413603452)
6 (413603452)
7 (413603452)
8 (413603452)
9 (413603452)
10 (413603452)
11 (413603452)
12 (413603452)
13 (413603452)
14 (413603452)
15 (413603452)
16 (413603452)
17 (413603452)
18 (413603452)
19 (413603452)
20 (413603452)
21 (413603452)
22 (413603452)
23 (413603452)
24 (413603452)
25 (413603452)


In [5]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(ll=l, p_phy=0.0005, infidelity=1e-9)
    print(l,volumes)

4 920000.0
5 920000.0
6 920000.0
7 920000.0
8 920000.0
9 920000.0
10 920000.0
11 920000.0
12 920000.0
13 920000.0
14 920000.0
15 920000.0
16 920000.0
17 920000.0
18 920000.0
19 920000.0
20 920000.0
21 920000.0
22 920000.0
23 920000.0
24 920000.0
25 920000.0


In [6]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(ll=l, p_phy=0.001, infidelity=1e-9)
    print(l,volumes)

4 201490346
5 201490346
6 201490346
7 201490346
8 201490346
9 201490346
10 201490346
11 201490346
12 201490346
13 201490346
14 201490346
15 201490346
16 201490346
17 201490346
18 201490346
19 201490346
20 201490346
21 201490346
22 201490346
23 201490346
24 201490346
25 201490346


In [7]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(ll=l, p_phy=0.0005, infidelity=1e-15)
    print(l,error_rates,volumes)   

4 5.514891577148439e-18 514729862.0
5 5.514891577148439e-18 514729862.0
6 5.514891577148439e-18 514729862.0
7 5.514891577148439e-18 514729862.0
8 5.514891577148439e-18 514729862.0
9 5.514891577148439e-18 514729862.0
10 5.514891577148439e-18 514729862.0
11 5.514891577148439e-18 514729862.0
12 5.514891577148439e-18 514729862.0
13 5.514891577148439e-18 514729862.0
14 5.514891577148439e-18 514729862.0
15 5.514891577148439e-18 514729862.0
16 5.514891577148439e-18 514729862.0
17 5.514891577148439e-18 514729862.0
18 5.514891577148439e-18 514729862.0
19 5.514891577148439e-18 514729862.0
20 5.514891577148439e-18 514729862.0
21 5.514891577148439e-18 514729862.0
22 5.514891577148439e-18 514729862.0
23 5.514891577148439e-18 514729862.0
24 5.514891577148439e-18 514729862.0
25 5.514891577148439e-18 514729862.0


In [8]:
for l in range(4, 26):
    error_rates, volumes = T_decomposition(ll=l, p_phy=0.001, infidelity=1e-15)
    print(l,error_rates, volumes)

4 0 inf
5 0 inf
6 0 inf
7 0 inf
8 0 inf
9 0 inf
10 0 inf
11 0 inf
12 0 inf
13 0 inf
14 0 inf
15 0 inf
16 0 inf
17 0 inf
18 0 inf
19 0 inf
20 0 inf
21 0 inf
22 0 inf
23 0 inf
24 0 inf
25 0 inf


In [2]:
if p_phy == 0.001:
    eps = [0.0004, 8e-05, 7e-06, 5e-06, 3e-06, 2e-07, 3e-08, 1e-08, 6e-09, 2e-09, 1e-09, 
         3.8850000000000016e-10, 3.304716553287983e-10, 1.2955e-12, 1.2178655692729767e-12, 4.405000000000001e-14,
         3.563400713436386e-14, 4.815000000000002e-15, 3.9081789802289295e-15, 1.2225000000000002e-15, 1.1252984389348027e-15, 
         1.3475000000000002e-16, 5.0750000000000023e-17, 4.03781224489796e-17]
    volume = [1000, 4000, 5000, 6000, 8000, 20000, 30000, 40000, 50000, 60000, 100000, 
         2199074, 2523632, 4685822, 5216768, 5827026, 6438000, 7131094, 7827712, 8609258, 9397136, 10272750, 10432750, 11317504]
elif p_phy == 0.0005:
    eps=[1e-5, 8e-7,3e-07, 2e-07, 1e-08, 1e-09, 4e-11, 3.7260546875000005e-12, 3.2233414127423824e-12, 1.186650390625e-12, 
         1.1583162379535146e-12,2.9983642578125002e-15, 2.91156640625e-15, 3.871166992187501e-17, 3.3973240312071337e-17, 
         7.554645996093752e-20,6.170946930280958e-20]
    volume=[2000, 3500, 4000, 4500, 8000.0, 10000.0, 20000.0, 1621006, 1888000, 2185074, 
            2509632, 3688250.0, 4144784.0, 
            4645822.0, 5176768.0,7051094.0, 7747712.0]

([[2e-07, 20000],
  [3e-08, 30000],
  [1e-08, 40000],
  [6e-09, 50000],
  [2e-09, 60000],
  [1e-09, 100000]],
 [[4.405000000000001e-14, 5827026],
  [3.563400713436386e-14, 6438000],
  [4.815000000000002e-15, 7131094]])

295049190.76237065